# OPTaaS - Advanced Options

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. You can get one <a href="https://optaas.ai/">here</a>.

## Connect to the OPTaaS server using your API Key

In [1]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

## Store additional data in a Task

This can be a JSON, array, string, number or boolean:

In [2]:
from mindfoundry.optaas.client.parameter import FloatParameter

task = client.create_task(
    title='My Task with User-defined Data', 
    parameters=[
        FloatParameter('x', minimum=0, maximum=5),
        FloatParameter('y', minimum=1, maximum=5),
    ],
    user_defined_data={
        'description': 'Lorem ipsum...',
        'tags': ['abc', 'defg']
    }
)

print(task.user_defined_data)

{'description': 'Lorem ipsum...', 'tags': ['abc', 'defg']}


## Warm start

If you've already tried some configurations, you can record the scores upfront, thus giving the optimizer a "warm start":

In [3]:
def scoring_function(x, y):
    return (x * y) - (x / y)

warm_start_values = [
    {'x': 0, 'y': 1},
    {'x': 1, 'y': 2},
    {'x': 0.5, 'y': 1.5},
]
for values in warm_start_values:
    task.add_user_defined_configuration(values, scoring_function(**values))

task.get_results()

[{ 'configuration': '971e36cf-4a67-4647-8f85-ab61468a897f',
   'id': 1765,
   'score': 0.0,
   'user_defined_data': None},
 { 'configuration': '811f2a7a-ad7c-4a56-b55d-599ce2821005',
   'id': 1766,
   'score': 1.5,
   'user_defined_data': None},
 { 'configuration': 'ea9a55a9-f152-473c-9e6c-0bb6e9a708ad',
   'id': 1767,
   'score': 0.4166666666666667,
   'user_defined_data': None}]

## Generate multiple configurations

This can be useful if you want to use parallel computation, i.e. generate several configurations and hand them out to separate workers to calculate scores.

In [4]:
number_of_workers = 5
configurations = task.generate_configurations(number_of_workers)
display(configurations)

[{ 'id': 'b294a8b5-d625-402e-8eb4-540a0e756062',
   'type': 'exploration',
   'values': {'x': 0.7381561015841065, 'y': 3.769275362896137}},
 { 'id': '5aeece38-ea5b-4f19-ba52-55f0fbd430eb',
   'type': 'exploration',
   'values': {'x': 0.8570314819084718, 'y': 4.551593966241775}},
 { 'id': '9b1e96cd-0aff-4d7c-828e-66ed405a46ae',
   'type': 'exploration',
   'values': {'x': 4.539677653466581, 'y': 4.174272699678086}},
 { 'id': '11f0821d-21e1-442a-9c49-b674c6c6259b',
   'type': 'default',
   'values': {'x': 2.5, 'y': 3.0}},
 { 'id': 'a46119ca-9518-4dad-a9a5-a0d46280c8d9',
   'type': 'exploration',
   'values': {'x': 2.9537119330383477, 'y': 3.7582400894566743}}]

## Store additional data in a Result

This can be a JSON, array, string, number or boolean:

In [5]:
for i, configuration in enumerate(configurations):
    score = scoring_function(**configuration.values)
    display(f'Worker {i} Score {score:.3f}')
    next_configuration = task.record_result(configuration, score, 
                                            user_defined_data={'worker': i})

'Worker 0 Score 2.586'

'Worker 1 Score 3.713'

'Worker 2 Score 17.862'

'Worker 3 Score 6.667'

'Worker 4 Score 10.315'

## Report an error

If you encountered an error while calculating the score for a configuration, you can report it:

In [6]:
error_configuration = task.generate_configurations(1)[0]
next_configuration = task.record_result(error_configuration, 
                                        error='Unexpected error: code 12345')
error_result = task.get_results()[-1]
print(f'Score: {error_result.score}  {error_result.error}')

Score: None  Unexpected error: code 12345


## Get best result and configuration

In [7]:
task.get_best_result_and_configuration()

({ 'configuration': '9b1e96cd-0aff-4d7c-828e-66ed405a46ae',
   'id': 1770,
   'score': 17.862315100248942,
   'user_defined_data': {'worker': 2}},
 { 'id': '9b1e96cd-0aff-4d7c-828e-66ed405a46ae',
   'type': 'exploration',
   'values': {'x': 4.539677653466581, 'y': 4.174272699678086}})

## Get top 5 results and configurations

In [8]:
for result in task.get_results(limit=5, best_first=True):
    configuration = task.get_configuration(result.configuration)
    display(f'Score: {result.score:.3f} for configuration: {configuration.values}')

"Score: 17.862 for configuration: {'x': 4.539677653466581, 'y': 4.174272699678086}"

"Score: 10.315 for configuration: {'x': 2.9537119330383477, 'y': 3.7582400894566743}"

"Score: 6.667 for configuration: {'x': 2.5, 'y': 3.0}"

"Score: 3.713 for configuration: {'x': 0.8570314819084718, 'y': 4.551593966241775}"

"Score: 2.586 for configuration: {'x': 0.7381561015841065, 'y': 3.769275362896137}"